# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [37]:
! pip install -U langchain

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [14]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [25]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""

math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

biology_template = """你是一位杰出的生物学家。你擅长解答生物学问题。
之所以如此卓越，是因为你能够将复杂的生物学现象分解为各自的基本部分，
先对这些基本部分进行解答，然后再综合它们来解答更广泛的生物学问题。

这是一个问题：
{input}"""


computer_template = """你是一位卓越的计算机专家。你擅长解决计算机相关的难题。 
之所以表现出色，是因为你能将复杂的技术问题分解为基础元素， 
首先针对这些元素找出解决方案，然后将它们结合起来，以此解决更全面的计算机科学问题。

现在来看一个问题：
{input}"""

chinese_literature_template = """你是一位汉语言文学的权威，深谙语言之美。
你以清晰简洁的方式解答各类汉语言文学问题，如诗词、文言文等。
遇到不明了的问题，你会真诚承认，并愿意深入挖掘知识的深度。
   

这是一个问题：
{input}"""

In [26]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": biology_template,
    },
    {
        "name": "汉语言文学",
        "description": "适用于回答汉语言文学问题",
        "prompt_template": chinese_literature_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算机问题",
        "prompt_template": computer_template,
    }
]

In [49]:
llm = OpenAI(api_key='sk-u1KVKPjooxeYYhsm318b42F4922047B8846c3f0bA33bA59c',base_url='https://api.xiaoai.plus/v1',model_name="gpt-3.5-turbo-instruct")

In [50]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [51]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [52]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [53]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [54]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物问题', '汉语言文学: 适用于回答汉语言文学问题', '计算机: 适用于回答计算机问题']


In [55]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
汉语言文学: 适用于回答汉语言文学问题
计算机: 适用于回答计算机问题


In [56]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [57]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
汉语言文学: 适用于回答汉语言文学问题
计算机: 适用于回答计算机问题

<< INPUT >>
{input}

<

In [58]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [59]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': '黑体辐射是什么？?'}
> Finished chain.
{'input': '黑体辐射是什么？?', 'text': '\n\n黑体辐射是一种物理现象，指的是处于热平衡状态的物体所发出的电磁波。根据普朗克定律，黑体辐射的强度与温度有关，温度越高，发射的波长越短，能量越高。黑体辐射是理解热力学和量子物理的重要概念，也有广泛的应用，例如在太阳能电池和红外线热像仪中。'}


In [60]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '40'}
> Finished chain.
{'input': '40', 'text': '个人参加了一场比赛，有几人获得了奖牌？\n\n首先，我们可以分解这个问题，将它分解成更小的组成部分：\n1. 有多少个奖牌？\n2. 每个奖牌分别是什么？\n3. 这些奖牌被颁发给了哪些人？\n\n然后，我们可以回答每个组成部分：\n1. 一共有三种奖牌：金牌、银牌和铜牌。\n2. 金牌是第一名获得的奖励，银牌是第二名获得的奖励，铜牌是第三名获得的奖励。\n3. 这些奖牌被颁发给了前三名的选手，即每种奖牌有一个获得者。\n\n最后，我们可以整合这些答案来回答更广泛的问题：\n一共'}


In [61]:
router_chain.verbose = True

In [62]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '黑洞是什么？'}
> Finished chain.
{'input': '黑洞是什么？', 'text': '\n黑洞是一种密度极高的天体，它的引力非常强大，甚至连光也无法逃逸。它的形成是由于某个恒星在死亡时，由于自身重力崩溃而形成的。在黑洞的中心，有一个称为“奇点”的点，其密度无限大，空间曲率也达到无限大，这也是为什么连光也无法逃逸的原因。黑洞有着许多奇特的特性和现象，仍然是物理学研究的热点之一。'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains

In [63]:
print(chain.invoke("解释一下DNA和RNA的区别是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
生物: {'input': 'DNA和RNA的区别'}
> Finished chain.
{'input': 'DNA和RNA的区别', 'text': '是什么？\n\nDNA和RNA是生物体内最重要的两种核酸分子。它们在结构、功能和作用方式上有着明显的差异。\n\n首先，DNA的全称是脱氧核糖核酸，RNA的全称是核糖核酸。它们的名字中都包含“核酸”这一词，是因为它们都由核苷酸构成。核苷酸包括磷酸、五碳糖和一种碱基，DNA和RNA的碱基有所不同。\n\n其次，DNA和RNA的五碳糖也不同。DNA的五碳糖是脱氧核糖，而RNA的五碳糖是核糖。\n\n最重要的区别在于它们的功能和作用方式。DNA是遗传物质的载体，它存储了生物体内'}


In [65]:
print(chain.invoke("计算机网络层是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
计算机: {'input': '计算机网络层是什么？'}
> Finished chain.
{'input': '计算机网络层是什么？', 'text': '\n计算机网络层是计算机网络体系结构中的一个重要组成部分，它负责处理数据包的路由选择和转发，以及实现网络间的通信服务。它位于网络体系结构的第三层，处于传输层和数据链路层之间。其主要功能包括网络地址分配、路由选择、流量控制、拥塞控制等。网络层使用IP协议来实现数据包的路由选择和转发，保证数据包能够顺利地从源主机传输到目的主机。同时，网络层还可以提供多种服务，如虚拟专用网（VPN）、负载均衡、安全隧道等。总的来说，网络层是构建计算机网络的重要组成部分，它使得不同的网络能够互相通信，实现信息的传输和共享。'}


In [66]:
print(chain.invoke("欲买桂花同载酒出自谁的诗词？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
None: {'input': '欲买桂花同载酒出自谁的诗词？'}
> Finished chain.
{'input': '欲买桂花同载酒出自谁的诗词？', 'history': '', 'text': ' 这首诗词的作者是唐代诗人李白。它被收录在李白的《九百年祭》中，是一首描写出游赏月的诗歌。它的首句是“欲买桂花同载酒，终不似，少年游”。这首诗描写了作者心中的理想，希望能和好友一起欣赏月光，享受青春的美好时光。'}
